### Importing libraries

In [1]:
import llama_index.core
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_parse import LlamaParse
import nest_asyncio; nest_asyncio.apply()

import json
import os
import re
from dotenv import load_dotenv
load_dotenv()

True

### Loading data and index

In [2]:
BP_threshold = 30
top_k = 24 # set how many chunks are given as context to GPT

PERSIST_DIR = f"./experiments_with_oracle/storage_SDR_semantic_chunking_various_thresholds/storage_SDR_sem_{BP_threshold}_text-embedding-ada-002"
LIST_OF_DOCS = ["raw_texts/"+f for f in os.listdir("Raw_texts")]

print(LIST_OF_DOCS)

storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
index = load_index_from_storage(storage_context)

['raw_texts/2-settext.pdf', 'raw_texts/3-settext.txt', 'raw_texts/4-settext.pdf', 'raw_texts/5-settext.pdf', 'raw_texts/9-settext.pdf']


###  Setting up RAG

In [3]:
model = "gpt-4o-mini"

Settings.llm = OpenAI(temperature=0.0, model=model)

def pingGPT(text_input):
   response = query_engine.query(text_input)
   return response.response

### Open quizzes

In [4]:
GPT_made_quizzes = ["JSON_quizzes/GPT-made_quizzes/"+f for f in os.listdir("./JSON_quizzes/GPT-made_quizzes") if f.endswith('.json')]
my_own_quizzes = ["JSON_quizzes/my_own_quizzes/"+f for f in os.listdir("./JSON_quizzes/my_own_quizzes") if f.endswith('.json')]
LIST_OF_QUIZZES = GPT_made_quizzes + my_own_quizzes

print(LIST_OF_QUIZZES)
print("number of quizzes:", len(LIST_OF_QUIZZES))

['JSON_quizzes/GPT-made_quizzes/itembank-2.json', 'JSON_quizzes/GPT-made_quizzes/itembank-3.json', 'JSON_quizzes/GPT-made_quizzes/itembank-4.json', 'JSON_quizzes/GPT-made_quizzes/itembank-5.json', 'JSON_quizzes/GPT-made_quizzes/itembank-9.json', 'JSON_quizzes/my_own_quizzes/itembank-2.json', 'JSON_quizzes/my_own_quizzes/itembank-3.json', 'JSON_quizzes/my_own_quizzes/itembank-4.json', 'JSON_quizzes/my_own_quizzes/itembank-5.json', 'JSON_quizzes/my_own_quizzes/itembank-9.json']
number of quizzes: 10


### The retriever with default prompt

In [5]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

# configure response synthesizer

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

### Full loop

In [ ]:
# save total number of questions and correct answers separately for my own quizzes and GPT-made
total_nquestions_own = 0
total_ncorrects_own = 0
total_nquestions_GPT = 0
total_ncorrects_GPT = 0

itembank_reports = [] # to write the itembank number and number of correct answers.
itembank_reports.append(f"breakpoint percentile: {BP_threshold}")
itembank_reports.append(f"top_k: {top_k}")

# Iterate over each question and get GPT's response
for doc in LIST_OF_QUIZZES:
    with open(doc, 'r', encoding='UTF-8') as file:
        itembank_JSON = json.load(file)
    
    itembank_number = re.search(r"\d[a|b]?(?:\-\d)?", doc).group()
    quiz_type = "own_questions" if re.search(r"my_own_quizzes", doc) else "GPT-made_questions" # returns None for GPT-made questions
    this_ncorrects = 0 # for correct answers in each separate itembank
    this_nquestions = 0

    print(f"----------processing itembank-{itembank_number}-{quiz_type}----------")

    for question in itembank_JSON:
        full_prompt = question['question'] + '\n' + '\n'.join(question['answers'])
        given_answer = pingGPT(full_prompt)  # Get the response from GPT
        question["GPT's response"] = given_answer  # Append the response to the question
        question["correct"] = given_answer==question["correct answer(s)"]

        if quiz_type == "own_questions":
            if given_answer==question["correct answer(s)"]: 
                total_ncorrects_own+=1
                this_ncorrects+=1
            total_nquestions_own+=1
            this_nquestions+=1
        else:
            if given_answer==question["correct answer(s)"]: 
                total_ncorrects_GPT+=1
                this_ncorrects+=1
            total_nquestions_GPT+=1
            this_nquestions+=1
        
    itembank_reports.append(f"itembank-{itembank_number}_{quiz_type}:\t\tCorrect answers {this_ncorrects} / total questions {this_nquestions} * 100 = {round(this_ncorrects/this_nquestions*100, 2)}%")

    # Save the updated JSON data to a new file
    with open(f'./results/improved_RAG_responses/semantic_chunking/RAG_{model}_responses-semantic_{BP_threshold}-{itembank_number}_{quiz_type}.json', 'w') as outfile:
        json.dump(itembank_JSON, outfile, indent=2)

itembank_reports.append(f"\nOverall correct answers for GPT-made {total_ncorrects_GPT} / total questions {total_nquestions_GPT} * 100 = {round(total_ncorrects_GPT/total_nquestions_GPT*100, 2)}%")
itembank_reports.append(f"Overall correct answers for own {total_ncorrects_own} / total questions {total_nquestions_own} * 100 = {round(total_ncorrects_own/total_nquestions_own*100, 2)}%\n")
itembank_reports.append("\n------------------------------------------------------------------------\n\n")

# save the prompt and number of correct questions
with open(f"./results/default_rag_{model}_semantic_chunking_results.txt", 'a', encoding='UTF-8') as report_file:
    report_file.write('\n'.join(itembank_reports))

----------processing itembank-2-GPT-made_questions----------
----------processing itembank-3-GPT-made_questions----------
----------processing itembank-4-GPT-made_questions----------
----------processing itembank-5-GPT-made_questions----------
----------processing itembank-9-GPT-made_questions----------
----------processing itembank-2-own_questions----------
----------processing itembank-3-own_questions----------
----------processing itembank-4-own_questions----------
----------processing itembank-5-own_questions----------
----------processing itembank-9-own_questions----------


: 

### Observe with Phoenix (not necessary to run)

In [ ]:
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={os.getenv('PHOENIX_API_KEY')}"
llama_index.core.set_global_handler("arize_phoenix", endpoint="https://llamatrace.com/v1/traces")

with open('./JSON_quizzes/my_own_quizzes/itembank-4.json', 'r', encoding='UTF-8') as file:
    questions_JSON = json.load(file)

for question in questions_JSON[18:19]:
    full_prompt = question['question'] + '\n' + '\n'.join(question['answers'])
    given_answer = pingGPT(full_prompt)  # Get the response from GPT
print(given_answer)